In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
import torch

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

In [5]:
# fig = plt.figure(figsize=(25, 25))
# sns.heatmap(train_df.corr(), yticklabels=False, cbar=False, cmap="viridis", vmax=0.8)

In [6]:
# fig = plt.figure(figsize=(20, 20))
# sns.heatmap(train_df.isnull(), yticklabels=False, cbar=False, cmap="viridis")

In [7]:
train_df.corr()["SalePrice"].sort_values()

ExterQual_TA       -0.589044
KitchenQual_TA     -0.519298
BsmtQual_TA        -0.452394
GarageFinish_Unf   -0.410608
MasVnrType_None    -0.374468
                      ...   
GarageArea          0.623431
GarageCars          0.640409
GrLivArea           0.708624
OverallQual         0.790982
SalePrice           1.000000
Name: SalePrice, Length: 290, dtype: float64

In [8]:
train_df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [9]:
train_df.fillna(train_df.median())
test_df.fillna(test_df.median())

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0,0,0,1,1,0,0,0,0,0
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,0,0,0,1,1,0,0,0,0,0
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,0,0,0,1,0,0,0,0,1,0


In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [11]:
att_train = train_df.columns
att_test = test_df.columns
not_included = att_train ^ att_test
Y_train = train_df["SalePrice"]
train_df.drop(not_included, axis=1, inplace=True)
# test_df.drop(not_included, axis=1, inplace=True)
X_train = train_df.drop("Id", axis=1)
X_test = test_df.drop("Id", axis=1)

/var/folders/_j/q21qb4mn0kv1nvcchfs_qh_r0000gn/T/ipykernel_15875/2449702125.py:3: FutureWarning: Index.__xor__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__xor__.  Use index.symmetric_difference(other) instead.
  not_included = att_train ^ att_test


In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
X_train.shape

(1460, 270)

In [14]:
X_test

array([[-0.87256276,  0.40989452,  0.11076257, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-0.87256276,  0.45108977,  0.37584985, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.07337496,  0.16272298,  0.33205282, ..., -0.11785113,
         0.4676514 , -0.30599503],
       ...,
       [-0.87256276,  3.70551504,  0.95042275, ..., -0.11785113,
        -2.13834494, -0.30599503],
       [ 0.66458604, -0.3316201 , -0.00759964, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.07337496,  0.16272298, -0.08918038, ..., -0.11785113,
         0.4676514 , -0.30599503]])

In [15]:
import lightning as L
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

In [16]:
class HouseModel(L.LightningModule):
    def __init__(self, scaler):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(270, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
        self.criterion = nn.HuberLoss()
        self.scaler = scaler
    
    def forward(self, x):
        x = self.net(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.net(x)
        loss = self.criterion(y_pred, y)
        abs_error = self.scaler.inverse_transform(abs(y_pred - y))
        self.log("loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        self.log("abs_error", abs_error, prog_bar=True, on_step=True, on_epoch=True)

        return {"loss", loss}
    
    def configure_optimizers(self):
        return optim.Adam(self.net.parameters(), lr=1e-3)

In [17]:


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)

train_ds = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(dataset=train_ds, shuffle=True, batch_size=32)
model = HouseModel(scaler)
early_stopping = EarlyStopping(monitor="train_loss", mode="min", patience=10)
tb_logger = TensorBoardLogger("results", name="my_model")

In [18]:
trainer = L.Trainer(max_epochs=100, callbacks=[early_stopping], logger=tb_logger,gpus=0)


/Users/ksterx/Development/miniforge3/envs/ai/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ksterx/Development/miniforge3/envs/ai/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [19]:
trainer.fit(model=model, train_dataloaders=train_dataloader)



  | Name      | Type       | Params
-----------------------------------------
0 | net       | Sequential | 21.6 K
1 | criterion | HuberLoss  | 0     
-----------------------------------------
21.6 K    Trainable params
0         Non-trainable params
21.6 K    Total params
0.086     Total estimated model params size (MB)
/Users/ksterx/Development/miniforge3/envs/ai/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/ksterx/Development/miniforge3/envs/ai/lib/python3.10/site-packages/lightning/pytorch/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (46) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower 

Training: 0it [00:00, ?it/s]

/Users/ksterx/Development/miniforge3/envs/ai/lib/python3.10/site-packages/torch/nn/modules/loss.py:988: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.